In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-53a99ce5-21b8-7840-769d-06e5ec79b5ee)


In [ ]:
!pip install transformers torch torchtext Pillow pickle matplotlib tqdm numpy requests time

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
from transformers import GPT2TokenizerFast, ViTImageProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset
from torchtext.data import get_tokenizer
import requests
import torch
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import time

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model_path = "/content/drive/MyDrive/ML Final Project/Models/vit-gpt2-image-captioning"
processor_path = "/content/drive/MyDrive/ML Final Project/Processor/vit-gpt2-image-captioning"
tokenizer_path = "/content/drive/MyDrive/ML Final Project/Tokenizer/vit-gpt2-image-captioning"

if not os.path.exists(model_path):
    model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    model.save_pretrained(model_path)

else:
    model = VisionEncoderDecoderModel.from_pretrained(model_path)

if not os.path.exists(processor_path):
    processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    processor.save_pretrained(processor_path)
else:
    processor = ViTImageProcessor.from_pretrained(processor_path)

if not os.path.exists(tokenizer_path):
    tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    tokenizer.save_pretrained(tokenizer_path)
else:
    processor = GPT2TokenizerFast.from_pretrained(tokenizer_path)

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

Removed shared tensor {'decoder.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [ ]:
def generate_captions(url, model = model):
    image = Image.open(requests.get(url, stream =True).raw).convert("RGB")
    pixel_values   = processor(image, return_tensors ="pt").pixel_values.to(device)
    # plt.imshow(np.asarray(image))
    # plt.show()

    generated_ids  = model.generate(
        pixel_values,
        do_sample=False,
        max_length=16,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        top_k=5,)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    #print(generated_text)
    return generated_text

url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"

generate_captions(url)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


'a woman in a blue shirt is riding on the back of a truck '

In [ ]:
import json
with open('/content/drive/MyDrive/ML Final Project/captions_val2014.json', 'r') as file:
    data = json.load(file)

beg = 0
threshold = 249
images_data = data.get('images', [])

# random.shuffle(images_data)
subset_size = len(images_data) // 5
selected_images = images_data[beg:subset_size]
#print(f"subset size: {subset_size}")
generated_captions = []

count = beg
for image_info in selected_images:
    start = time.time()
    image_url = image_info.get('url')

    image_id = image_info.get('id')
    resp = requests.get(image_url, stream=True)
    if not resp.ok:
        continue
    else:
        generated_text = generate_captions(image_url, greedy = False)

    image_id = image_info.get('id')
    generated_captions.append({"image_id": image_id, "caption": generated_text})
    count = count+1
    end = time.time()
    print(f"Processed Image: {count}/{threshold+1} \t Time Taken:{end-start}s")
    with open('/content/drive/MyDrive/ML Final Project/captions_vit-gpt2-image-captioning.json', 'w') as result_file:
        json.dump(generated_captions, result_file)
    if count > threshold:
        break

Processed Image: 1/250 	 Time Taken:0.4619896411895752s
Processed Image: 2/250 	 Time Taken:0.4789316654205322s
Processed Image: 3/250 	 Time Taken:0.46796441078186035s
Processed Image: 4/250 	 Time Taken:0.49227476119995117s
Processed Image: 5/250 	 Time Taken:0.4300806522369385s
Processed Image: 6/250 	 Time Taken:0.4930143356323242s
Processed Image: 7/250 	 Time Taken:0.5159626007080078s
Processed Image: 8/250 	 Time Taken:0.46417999267578125s
Processed Image: 9/250 	 Time Taken:0.42238807678222656s
Processed Image: 10/250 	 Time Taken:0.501258373260498s
Processed Image: 11/250 	 Time Taken:0.37564682960510254s
Processed Image: 12/250 	 Time Taken:0.4434492588043213s
Processed Image: 13/250 	 Time Taken:0.45162510871887207s
Processed Image: 14/250 	 Time Taken:0.5198285579681396s
Processed Image: 15/250 	 Time Taken:0.39969539642333984s
Processed Image: 16/250 	 Time Taken:0.44908785820007324s
Processed Image: 17/250 	 Time Taken:0.45607852935791016s
Processed Image: 18/250 	 Time T